<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/cliquetoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for formatting rates/vols if needed (from previous examples)
def format_rate(r):
    return f"{r * 100:.4f}%"
def format_vol(v):
    return f"{v * 100:.4f}%"

# Helper to mimic REPORT_FAILURE macro for simple chooser
def report_failure_simple(test_case_instance, greekName, choosingDate,
                          exercise, s, q, r, today, v,
                          expected, calculated, tolerance):
    msg = (
        f"{exercise} " # Needs exerciseTypeToString equivalent if available
        f"Simple Chooser option with \n"
        f"    spot value: {s}\n"
        f"    dividend yield:   {format_rate(q)}\n"
        f"    risk-free rate:   {format_rate(r)}\n"
        f"    reference date:   {today}\n"
        # Exercise doesn't directly have lastDate() in Python? Need to extract from exercise obj
        # f"    maturity:         {exercise.lastDate()}\n"
        f"    choosing date:    {choosingDate}\n"
        f"    volatility:       {format_vol(v)}\n\n"
        f"    expected   {greekName}: {expected:.6f}\n"
        f"    calculated {greekName}: {calculated:.6f}\n"
        f"    error:            {abs(expected-calculated):.6f}\n"
        f"    tolerance:        {tolerance}"
    )
    test_case_instance.fail(msg)

class ChooserOptionTests(unittest.TestCase):

    def setUp(self):
        """Set up the calculation date before each test."""
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Use a fixed date for consistency, or use today's date and set it
        # The C++ tests use Date::todaysDate() or Settings::instance().evaluationDate()
        # which might rely on the fixture setting it. Let's fix it here.
        self.today = ql.Date(15, ql.May, 2024) # Example fixed date
        ql.Settings.instance().evaluationDate = self.today

    def tearDown(self):
        """Restore the calculation date after each test."""
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testAnalyticSimpleChooserEngine(self):
        """Testing analytic simple chooser option."""
        print("Testing analytic simple chooser option...")

        # Market data setup mirroring C++ test
        dc = ql.Actual360()
        today = ql.Settings.instance().evaluationDate # Use the date set in setUp

        spot_q = ql.SimpleQuote(50.0)
        qRate_q = ql.SimpleQuote(0.0)
        rRate_q = ql.SimpleQuote(0.08)
        vol_q = ql.SimpleQuote(0.25)

        spot_h = ql.QuoteHandle(spot_q)
        qTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(qRate_q), dc))
        rTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(rRate_q), dc))
        # Assuming TARGET calendar if not specified, similar to previous examples
        volTS_h = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(vol_q), dc))

        stochProcess = ql.BlackScholesMertonProcess(spot_h, qTS_h, rTS_h, volTS_h)
        engine = ql.AnalyticSimpleChooserEngine(stochProcess)

        # Option details
        strike = 50.0
        exerciseDate = today + ql.Period(180, ql.Days)
        exercise = ql.EuropeanExercise(exerciseDate)
        choosingDate = today + ql.Period(90, ql.Days)

        option = ql.SimpleChooserOption(choosingDate, strike, exercise)
        option.setPricingEngine(engine)

        # Verification
        calculated = option.NPV()
        expected = 6.1071
        tolerance = 3e-5

        if abs(calculated - expected) > tolerance:
             report_failure_simple(self, "value", choosingDate,
                                   "European", spot_q.value(),
                                   qRate_q.value(), rRate_q.value(), today,
                                   vol_q.value(), expected, calculated, tolerance)
        # Also include standard assert for typical test runners
        self.assertAlmostEqual(calculated, expected, delta=tolerance)


    def testAnalyticComplexChooserEngine(self):
        """Testing analytic complex chooser option."""
        print("Testing analytic complex chooser option...")

        # Market data setup
        dc = ql.Actual360()
        today = ql.Date().todaysDate() # Uses actual current date, might differ from setUp's fixed date
        # Let's override to use the fixed date from setUp for consistency
        today = self.today
        ql.Settings.instance().evaluationDate = today # Ensure it's set for this test too

        spot_q = ql.SimpleQuote(50.0)
        qRate_q = ql.SimpleQuote(0.05)
        rRate_q = ql.SimpleQuote(0.10)
        vol_q = ql.SimpleQuote(0.35)

        spot_h = ql.QuoteHandle(spot_q)
        qTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(qRate_q), dc))
        rTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(rRate_q), dc))
        volTS_h = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(vol_q), dc))

        stochProcess = ql.BlackScholesMertonProcess(spot_h, qTS_h, rTS_h, volTS_h)
        engine = ql.AnalyticComplexChooserEngine(stochProcess)

        # Option details
        callStrike = 55.0
        putStrike = 48.0
        choosingDate = today + ql.Period(90, ql.Days)
        callExerciseDate = choosingDate + ql.Period(180, ql.Days)
        putExerciseDate = choosingDate + ql.Period(210, ql.Days)

        callExercise = ql.EuropeanExercise(callExerciseDate)
        putExercise = ql.EuropeanExercise(putExerciseDate)

        option = ql.ComplexChooserOption(choosingDate, callStrike, putStrike, callExercise, putExercise)
        option.setPricingEngine(engine)

        # Verification
        calculated = option.NPV()
        expected = 6.0508
        tolerance = 1e-4
        error = abs(calculated - expected)

        self.assertLessEqual(error, tolerance,
                             msg=(f"Failed to reproduce complex chooser option value:\n"
                                  f"  Expected:   {expected:.6f}\n"
                                  f"  Calculated: {calculated:.6f}\n"
                                  f"  Error:      {error:.6f}\n"
                                  f"  Tolerance:  {tolerance}"))

if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)